In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [3]:
 #2 very important preprocessing is Resizing and Rescaling
batch_size=2

training_datagenarator= ImageDataGenerator(rescale=1./255,horizontal_flip=True,
   vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2)

In [9]:
#train=training_datagenarator.flow_from_directory('D:/Project/Train',
                                               # target_size=(224, 224),color_mode='rgb',
                                       #class_mode='categorical', batch_size=batch_size)

validation=training_datagenarator.flow_from_directory('D:/Project/try',
                                                target_size=(224, 224),color_mode='rgb',
                                       save_to_dir='D:/Project/try',class_mode='categorical',save_prefix='U',save_format='jpeg', batch_size=batch_size)
for inputs,outputs in validation:
  continue


Found 41 images belonging to 2 classes.


KeyboardInterrupt: 

In [18]:
#img = load_img('D:/Project/Train/fire/IMG_20220501_162053vf160688.jpeg')  # this is a PIL image
#x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
#x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
#i = 0
#for batch in datagen.flow(x, batch_size=1,
                          #save_to_dir='D:/Project/aug', save_prefix='nonfire', save_format='jpeg'):
    #i += 1
    #if i > 10:
       # break


In [19]:
#batch_size =16

In [30]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.layers import Dense, Activation,Flatten

In [27]:
mobile = tf.keras.applications.mobilenet.MobileNet()

mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [28]:
from tensorflow.keras.models import Model

In [31]:
#Deleting the last 5 layers and replacing it with a dense softmax layer consisting of 2 nodes: Fire and Non-fire
x = mobile.layers[-2].output
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=mobile.input, outputs=predictions)

In [32]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [33]:
#This is done because we will retrain only the last three layers
for layer in model.layers[:-4]:
    layer.trainable = False

In [36]:
model.compile(Adam(learning_rate=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
model.fit_generator(train, steps_per_epoch = 5, validation_data=validation, validation_steps=5, epochs=1)

C:\Users\mamoo\AppData\Local\Temp\ipykernel_7956\4007026727.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train, steps_per_epoch = 5, validation_data=validation, validation_steps=5, epochs=1)


5/5 [==============================] - 18s 2s/step - loss: 5.4059 - accuracy: 0.7250 - val_loss: 5.3931 - val_accuracy: 0.8250


In [38]:
model.save_weights('first_try.h5')

In [42]:
pip install opencv-python

  Using cached opencv_python-4.5.5.64-cp36-abi3-win_amd64.whl (35.4 MB)



In [43]:
import cv2
import os
import glob


In [66]:
img_dir = 'D:/Project/Valid/fire' # Enter Directory of all images 
data_path = os.path.join(img_dir,'*g')
files = glob.glob(data_path)
data = []
for f1 in files:
    img = cv2.imread(f1)
    data.append(img)

    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

i = 0
path, dirs, files = next(os.walk('D:/Project/Valid/fire'))
file_count = len(files) #to find number of files in folder

for batch in datagen.flow (x, batch_size=19, save_to_dir =r'D:/Project/Valid',save_prefix="a",save_format='jpg'):
    i+=1
    if i==20:
      break

In [71]:
PATH_TO_IMAGES = r'D:/Project/Valid'

# Specify whatever augmentation methods you want to use here
train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.3,
        height_shift_range=0.3, 
        shear_range=0.2, 
        zoom_range=0.2,
        horizontal_flip=True, 
        vertical_flip=True,
        fill_mode='nearest')



In [72]:
train_generator = train_datagen.flow_from_directory(
        PATH_TO_IMAGES,
        target_size=(224, 224),
        batch_size=19,
        save_to_dir='D:/Project/Valid',
        class_mode='categorical')

# Use the generator by calling .next()
train_generator.next()



Found 0 images belonging to 0 classes.


(array([], shape=(0, 224, 224, 3), dtype=float32),
 array([], shape=(0, 0), dtype=float32))

In [64]:
images = [cv2.imread(file) for file in glob.glob("D:/Project/Valid*.jpg")]

In [65]:
path = glob.glob("D:/Project/Valid/fire*.jpg")
cv_img = []
for img in path:
    n = cv.imread(img)
    cv_img.append(n)

In [68]:
datagen = ImageDataGenerator(rotation_range =15, 
                         width_shift_range = 0.2, 
                         height_shift_range = 0.2,  
                         rescale=1./255, 
                         shear_range=0.2, 
                         zoom_range=0.2, 
                         horizontal_flip = True, 
                         fill_mode = 'nearest', 
                         data_format='channels_last', 
                         brightness_range=[0.5, 1.5]) 

 


path = glob.glob("D:/Project/Valid/fire*.jpg")
cv_img = []
cv_img.nparray()
for img in path:
    n = cv.imread(img)
    cv_img.append(n)
#for f1 in files:
    #img = cv2.imread(f1)
    #data.append(img)

x = img_to_array(cv_img)
x = x.reshape((1,) + x.shape)

i = 0
path, dirs, files = next(os.walk("D:/Project/Valid/*.jpg"))
#file_count = len(files) #to find number of files in folder

for batch in datagen.flow (x, batch_size=1, save_to_dir =r"D:/Project/Valid/*.jpg",save_prefix="a",save_format='jpg'):
    i+=1
    if i==file_count:
      break

AttributeError: 'list' object has no attribute 'nparray'

In [ ]:
datagen = ImageDataGenerator(rotation_range =15, 
                     width_shift_range = 0.2, 
                     height_shift_range = 0.2,  
                     rescale=1./255, 
                     shear_range=0.2, 
                     zoom_range=0.2, 
                     horizontal_flip = True,) 

imgs = os.listdir("D:/Project/Valid/fire")

for img in imgs:
    img=cv2.imread("D:/Project/Valid/fire"+"\\"+img)
    x = img_to_array(img)
    x = x.reshape((1,) + x.shape)

    i = 0
    for batch in datagen.flow (x, batch_size=1, save_to_dir =r"D:/Project/Valid", save_prefix ='p2', save_format='jpg'):
        i+=1
        if i>10:
            break

In [94]:
dataset=ImageDataGenerator(rescale=1./255,horizontal_flip=True,
   vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2)
generator = dataset.flow_from_directory('D:/Project/Valid',target_size=(224,224),save_to_dir='D:/Project/Valid',class_mode='categorical',save_prefix='N',save_format='jpeg',batch_size=1)
for inputs in generator:
  continue


Found 129 images belonging to 2 classes.


KeyboardInterrupt: 